## Task 1

The general Bellman equation is:

$V^\pi(s)=r(s,a)+\gamma V^\pi(\delta (s,a))$

For our policy $\pi$ as defined in the question:

$r(FACEBOOK,quit)=-1$

$r(CLASS1,study)=-2$

$r(CLASS2,study)=-2$

$r(CLASS3,pub)=1$

$V^\pi(FACEBOOK)=-1+\gamma V^\pi(CLASS1)$

$V^\pi(CLASS1)=-2+\gamma V^\pi(CLASS2)$

$V^\pi(CLASS2)=-2+\gamma V^\pi(CLASS3)$

$V^\pi(CLASS3)=0.2(1+\gamma V^\pi(CLASS1))+0.4(1+\gamma V^\pi(CLASS2))+0.4(1+\gamma V^\pi(CLASS3))$

Solving this linear system gives us: $V^\pi(FACEBOOK)=-6.15041, V^\pi(CLASS1)=-5.72268, V^\pi(CLASS2)=-4.13631, V^\pi(CLASS3)=-2.37368$


## Task 2

Bellman optimality condition is:

$V^*(FACEBOOK)= \max{[r(FACEBOOK,a)+\gamma V^*(\delta (FACEBOOK,a))]} = max{[-1+\gamma V^*(CLASS1), -1+\gamma V^*(FACEBOOK)]}$

$V^*(CLASS1)= \max{[r(CLASS1,a)+\gamma V^*(\delta (CLASS1,a))]} = max{[-2+\gamma V^*(CLASS2), -1+\gamma V^*(FACEBOOK)]}$

$V^*(CLASS2)= \max{[r(CLASS2,a)+\gamma V^*(\delta (CLASS2,a))]} = max{[-2+\gamma V^*(CLASS3), \gamma V^*(SLEEP)]}$

$V^*(CLASS3)= \max{[r(CLASS3,a)+\gamma V^*(\delta (CLASS3,a))]} = max{[10+\gamma V^*(SLEEP), 0.2(1+\gamma V^*(CLASS1))+0.4(1+\gamma V^*(CLASS2))+0.4(1+\gamma V^*(CLASS3))]}$

In [1]:
%matplotlib inline
# tells matplotlib to display plots in notebook
import numpy as np
import matplotlib.pyplot as plt

In [2]:
STATES = ['FACEBOOK','CLASS1','CLASS2','CLASS3','SLEEP'] # we don't include state 'SLEEP' since there are no actions that can be taken at the goal state
ACTIONS = ['facebook','quit','study','pub','sleep']

# if k actions are available at state S, then this returns an array of size k, where each element consists of:
# the action name, the immediate reward of taking that action, and a list of resultant states and the corresponding probabilities for them
def get_state_profile(state):
  if state=='FACEBOOK':
    return [['quit', -1, 'CLASS1', 1], ['facebook', -1, 'FACEBOOK', 1]]
  if state=='CLASS1':
    return [['study', -2, 'CLASS2', 1], ['facebook', -1, 'FACEBOOK', 1]]
  if state=='CLASS2':
    return [['study', -2, 'CLASS3', 1], ['sleep', 0, 'SLEEP', 1]]
  if state=='CLASS3':
    return [['study', 10, 'SLEEP', 1], ['pub', 1, 'CLASS1', 0.2, 'CLASS2', 0.4, 'CLASS3', 0.4]]
  if state=='SLEEP':
    return [[]]

def fix_default_q(q_table):
  val = np.NINF # just refers to invalid values
  q_table[0,2:]=val # disable all actions besides 'facebook', 'quit' for state FACEBOOK

  q_table[1,1]=val # disable all actions besides 'facebook', 'study' for state CLASS1
  q_table[1,3:]=val

  q_table[2,0:2]=val # disable all actions besides 'sleep', 'study' for state CLASS2
  q_table[2,3]=val

  q_table[3,0:2]=val
  q_table[3,4]=val # disable all actions besides 'pub', 'study' for state CLASS3

  q_table[4][:]=0

  return q_table


V_k = np.ones(shape=(len(STATES))) # randomly initialise as all 0's
gamma = 0.9 # discount factor
threshold = 0.01
counter = 0
while True:
  V_next = np.zeros(shape=(len(STATES)))
  Q = np.zeros(shape=(len(STATES), len(ACTIONS))) # Q(s, a) table stored

  for state_num in range(len(STATES)): # iterate through all states
    state_profile = get_state_profile(STATES[state_num])

    for action_num in range(len(state_profile)): # iterate through all possible actions at each state
      if STATES[state_num]=='SLEEP': # don't do computations for state 'SLEEP'
        break
      this_action_profile = state_profile[action_num]
      action = this_action_profile[0]
      reward = this_action_profile[1] # since reward is deterministic

      for index in range(2,len(state_profile[action_num]),2):
        # Q(s,a) = sum (R(s, a) + gamma*V_k) over all possible resultant states
        prob = this_action_profile[index+1]
        nextstate = STATES.index(this_action_profile[index])
        Q[state_num][ACTIONS.index(action)] += prob*(reward+gamma*V_k[nextstate])
    V_next[state_num] = np.max(Q[state_num])
  
  counter += 1
  delta = np.linalg.norm(V_k - V_next, ord=1)
  V_k = V_next
  print("No. of iterations: {} | V_k: {} | delta: {}" .format(counter, V_k, delta))
  if delta <= threshold:
    break
    
# output policy
print("\n=================\nFinal Q table:")
Q = fix_default_q(Q)
print(Q)
print("\n=================\nOptimal Policy:")
best_actions = np.argmax(Q, axis=1)
# optimal policy should be quit, study, study, study 
for state_num in range(len(STATES)-1):
  print("STATE: {} | ACTION: {}" .format(STATES[state_num], ACTIONS[best_actions[state_num]]))


No. of iterations: 1 | V_k: [ 0.   0.   0.9 10.9  0. ] | delta: 13.0
No. of iterations: 2 | V_k: [ 0.    0.    7.81 10.    0.  ] | delta: 7.8100000000000005
No. of iterations: 3 | V_k: [ 0.     5.029  7.    10.     0.   ] | delta: 5.839000000000001
No. of iterations: 4 | V_k: [ 3.5261  4.3     7.     10.      0.    ] | delta: 4.255100000000001
No. of iterations: 5 | V_k: [ 2.87  4.3   7.   10.    0.  ] | delta: 0.6561000000000003
No. of iterations: 6 | V_k: [ 2.87  4.3   7.   10.    0.  ] | delta: 0.0

Final Q table:
[[ 1.583  2.87    -inf   -inf   -inf]
 [ 1.583   -inf  4.3     -inf   -inf]
 [  -inf   -inf  7.      -inf  0.   ]
 [  -inf   -inf 10.     7.894   -inf]
 [ 0.     0.     0.     0.     0.   ]]

Optimal Policy:
STATE: FACEBOOK | ACTION: quit
STATE: CLASS1 | ACTION: study
STATE: CLASS2 | ACTION: study
STATE: CLASS3 | ACTION: study


The policy our value iteration algorithm produced indeed matches that of the intuitive optimal policy (intuitively, we should try to get to CLASS3 as quickly as possible to claim the +10 reward, and since every connection between the other states involves a negative number, we want to do it as fast as possible. (While CLASS2 -> SLEEP has an immediate reward of 0, the discounted reward of 10 available at CLASS3 far outweighs it. Same with taking action pub at CLASS3 - while the reward of +1 can be claimed infinite times, it is outweighed by the fact if we don't end up in CLASS3, it requires at minimum another -2 to get back to CLASS3.)

It can be verified that $V^\pi(FACEBOOK)=2.87, V^\pi(CLASS1)=4.3, V^\pi(CLASS2)=7, V^\pi(CLASS3)=10$ (assuming $V^\pi(SLEEP)=0$) is indeed the exact solution to the policy we obtained.

## Task 3


In [3]:
Q_count = 0
Q_new = np.array([[-1, -1, 0, 0, 0],
                  [-1.9, 0, -2, 0, 0],
                  [0, 0, -1.1, 0, 0],
                  [0, 0, 10, 1, 0],
                  [0, 0, 0, 0, 0]]).astype("float32")

Q_new = fix_default_q(Q_new)
print("==== Initial table: ====")
print(Q_new)

def update_q(state, next_state, action, alpha, gamma):
  global Q_new
  global Q_count
  state_profile = get_state_profile(state)
  for transition in state_profile:
    if transition[0]==action:
      rsa = transition[1]
      qsa = Q_new[STATES.index(state), ACTIONS.index(action)] # old Q(s, a) value
      new_q = qsa + alpha * (rsa + gamma * max(Q_new[STATES.index(next_state), :]) - qsa) # update rule
      Q_new[STATES.index(state), ACTIONS.index(action)] = new_q
      break
  Q_count+=1
  print("===== Q table after {} iterations ====" .format(Q_count))
  print(Q_new)
  

update_q('CLASS3','CLASS3','pub',1,gamma)
update_q('CLASS3','CLASS2','pub',1,gamma)
update_q('CLASS2','CLASS3','study',1,gamma)
update_q('CLASS3','CLASS1','pub',1,gamma)
update_q('CLASS1','CLASS2','study',1,gamma)
update_q('CLASS2','SLEEP','sleep',1,gamma)

==== Initial table: ====
[[-1.  -1.  -inf -inf -inf]
 [-1.9 -inf -2.  -inf -inf]
 [-inf -inf -1.1 -inf  0. ]
 [-inf -inf 10.   1.  -inf]
 [ 0.   0.   0.   0.   0. ]]
===== Q table after 1 iterations ====
[[-1.  -1.  -inf -inf -inf]
 [-1.9 -inf -2.  -inf -inf]
 [-inf -inf -1.1 -inf  0. ]
 [-inf -inf 10.  10.  -inf]
 [ 0.   0.   0.   0.   0. ]]
===== Q table after 2 iterations ====
[[-1.  -1.  -inf -inf -inf]
 [-1.9 -inf -2.  -inf -inf]
 [-inf -inf -1.1 -inf  0. ]
 [-inf -inf 10.   1.  -inf]
 [ 0.   0.   0.   0.   0. ]]
===== Q table after 3 iterations ====
[[-1.  -1.  -inf -inf -inf]
 [-1.9 -inf -2.  -inf -inf]
 [-inf -inf  7.  -inf  0. ]
 [-inf -inf 10.   1.  -inf]
 [ 0.   0.   0.   0.   0. ]]
===== Q table after 4 iterations ====
[[-1.   -1.    -inf  -inf  -inf]
 [-1.9   -inf -2.    -inf  -inf]
 [ -inf  -inf  7.    -inf  0.  ]
 [ -inf  -inf 10.   -0.71  -inf]
 [ 0.    0.    0.    0.    0.  ]]
===== Q table after 5 iterations ====
[[-1.   -1.    -inf  -inf  -inf]
 [-1.9   -inf  4.3   -